In [1]:
# libraries
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Karthick\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Karthick\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:

# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

64 documents
12 classes ['AccidentLevel', 'Compliment', 'Exit', 'bot1', 'goodbye', 'greetings', 'name', 'name1', 'noanswer', 'thanks', 'who', 'yes']
73 unique lemmatized words ["'re", "'s", ',', 'a', 'accident', 'am', 'amazing', 'are', 'awesome', 'best', 'bot', 'buddy', 'bye', 'cya', 'day', 'delight', 'differenet', 'for', 'good', 'goodbye', 'great', 'haroo', 'have', 'hello', 'help', 'helped', 'helpful', 'helping', 'hey', 'hi', 'holla', 'i', 'impressed', 'is', 'job', 'later', 'learning', 'leaving', 'level', 'lot', 'love', 'me', 'my', 'name', 'ney', 'nice', 'no', 'okay', 'quite', 'reliable', 's', 'see', 'thank', 'thanks', 'thankyou', 'that', 'the', 'there', 'ton', 'too', 'true', 'wassup', 'what', 'who', 'will', 'work', 'wow', 'ya', 'yaw', 'yep', 'yes', 'you', 'your']


In [3]:

# training initializer
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

# actual training
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# for choosing an optimal number of training epochs to avoid underfitting or overfitting use an early stopping callback to keras
# based on either accuracy or loos monitoring. If the loss is being monitored, training comes to halt when there is an 
# increment observed in loss values. Or, If accuracy is being monitored, training comes to halt when there is decrement observed in accuracy values.

# from keras import callbacks 
# earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
# callbacks =[earlystopping]

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")

Training data created
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               9472      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 12)                780       
                                                                 
Total params: 18,508
Trainable params: 18,508
Non-trainable params: 0
_________________________________________________________________


C:\Users\Karthick\AppData\Roaming\Python\Python38\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
13/13 [==============================] - 1s 3ms/step - loss: 2.5017 - accuracy: 0.1406
Epoch 2/200
13/13 [==============================] - 0s 2ms/step - loss: 2.3715 - accuracy: 0.2188
Epoch 3/200
13/13 [==============================] - 0s 2ms/step - loss: 2.3012 - accuracy: 0.2656
Epoch 4/200
13/13 [==============================] - 0s 1ms/step - loss: 2.2381 - accuracy: 0.2344
Epoch 5/200
13/13 [==============================] - 0s 2ms/step - loss: 2.1474 - accuracy: 0.2500
Epoch 6/200
13/13 [==============================] - 0s 2ms/step - loss: 2.0329 - accuracy: 0.2812
Epoch 7/200
13/13 [==============================] - 0s 1ms/step - loss: 2.0885 - accuracy: 0.2969
Epoch 8/200
13/13 [==============================] - 0s 2ms/step - loss: 1.9544 - accuracy: 0.3594
Epoch 9/200
13/13 [==============================] - 0s 2ms/step - loss: 1.8933 - accuracy: 0.3438
Epoch 10/200
13/13 [==============================] - 0s 2ms/step - loss: 1.8748 - accuracy: 0.3125
Epoch 11/

In [4]:
_, train_accuracy = model.evaluate(train_x, train_y, batch_size=8, verbose=0)

print('train accuracy: %.2f' % (train_accuracy*100))

train accuracy: 93.75
